In [1]:
import tensorflow as tf
import keras
from keras.preprocessing.image import load_img, img_to_array, save_img

# 변환하려는 이미지 경로
target_image_path = './data/test.jpg'
# 스타일 이미지 경로
style_reference_image_path = './data/images.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

Using TensorFlow backend.


In [2]:
print(target_image_path)
print(width)
print(height)
print(img_width)
# !pip install keras==2.2.4
keras.__version__

./data/test.jpg
226
305
296


'2.2.4'

In [3]:

import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더합니다
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:

from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합칩니다
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듭니다.
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드합니다
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료.')











모델 로드 완료.


In [5]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [6]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [7]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [8]:
# 층 이름과 활성화 텐서를 매핑한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 콘텐츠 손실에 사용할 층
content_layer = 'block5_conv2'
# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의합니다
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

In [9]:
# 손실에 대한 생성된 이미지의 그래디언트를 구합니다
grads = K.gradients(loss, combination_image)[0]

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체입니다
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'change'
iterations = 50

# 뉴럴 스타일 트랜스퍼의 손실을 최소화하기 위해 생성된 이미지에 대해 L-BFGS 최적화를 수행합니다
# 초기 값은 타깃 이미지입니다
# scipy.optimize.fmin_l_bfgs_b 함수가 벡터만 처리할 수 있기 때문에 이미지를 펼칩니다.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('현재 손실 값:', min_val)
    # 생성된 현재 이미지를 저장합니다
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_%d.png' % i
    save_img('./003/'+fname, img)
    end_time = time.time()
    print('저장 이미지: ', fname)
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0
현재 손실 값: 2813605000.0
저장 이미지:  change_0.png
0 번째 반복 완료: 10s
반복 횟수: 1
현재 손실 값: 1082385200.0
저장 이미지:  change_1.png
1 번째 반복 완료: 7s
반복 횟수: 2
현재 손실 값: 705731300.0
저장 이미지:  change_2.png
2 번째 반복 완료: 7s
반복 횟수: 3
현재 손실 값: 533085200.0
저장 이미지:  change_3.png
3 번째 반복 완료: 7s
반복 횟수: 4
현재 손실 값: 432747140.0
저장 이미지:  change_4.png
4 번째 반복 완료: 7s
반복 횟수: 5
현재 손실 값: 362423740.0
저장 이미지:  change_5.png
5 번째 반복 완료: 7s
반복 횟수: 6
현재 손실 값: 316345380.0
저장 이미지:  change_6.png
6 번째 반복 완료: 7s
반복 횟수: 7
현재 손실 값: 285222750.0
저장 이미지:  change_7.png
7 번째 반복 완료: 7s
반복 횟수: 8
현재 손실 값: 259099330.0
저장 이미지:  change_8.png
8 번째 반복 완료: 7s
반복 횟수: 9
현재 손실 값: 240172430.0
저장 이미지:  change_9.png
9 번째 반복 완료: 7s
반복 횟수: 10
현재 손실 값: 223464580.0
저장 이미지:  change_10.png
10 번째 반복 완료: 7s
반복 횟수: 11
현재 손실 값: 209862880.0
저장 이미지:  change_11.png
11 번째 반복 완료: 7s
반복 횟수: 12
현재 손실 값: 198538200.0
저장 이미지:  change_12.png
12 번째 반복 완료: 7s
반복 횟수: 13
현재 손실 값: 189770240.0
저장 이미지:  change_13.png
13 번째 반복 완료: 7s
반복 횟수: 14
현재 손실 값: 180321440.0
저장 이미지:  change_